In [77]:
import pandas as pd
from google.cloud import bigquery
from dotenv import load_dotenv
load_dotenv()

True

In [78]:
client = bigquery.Client()

In [79]:
bc = client.query("SELECT * FROM members.buk_cash WHERE role != 'parent'").to_dataframe()

E0000 00:00:1768206737.642936  943759 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [60]:
bc.head()

,id,custom_id,email,role,first_name,last_name
0,abba7e51-6bc1-401b-997d-b727e64e0aa2,NaN,vebjorn.risa2@gmail.com,admin,ZDeveloper,Admin
1,2655170d-6cac-4d40-bf21-25925320ba54,NaN,tim_baltz@hotmail.no,admin,Tim,Baltzersen
2,0f0500c2-5765-47cc-878b-833c1fb866e1,NaN,minahauk@gmail.com,admin,Mina,Bratlie
3,462e8c1d-fabd-43f0-bdae-d6888bbeef4f,NaN,sigvard@sibr.no,admin,Sigvard,Bratlie (admin)
4,6c62fefd-73bb-4683-9d88-3219d7c82d21,NaN,mathilde.ode@gmail.com,admin,Mathilde,Ekholdt


In [61]:
t.head()

,person_id,display_name,email,role,team
0,41647,michelle helen pfeifer,pfeifer.mhp@gmail.com,ansvarlig hjelpementor,blue
1,<NA>,sigurd baltzersen,sigbaltz@hotmail.com,ansvarlig mentor,blue
2,<NA>,norah knutsen,norah.knutsen@outlook.com,ansvarlig mentor,blue
3,46663,emilio seth baumann,baumannemilio04@gmail.com,genf,blue
4,45966,william bekkevold,williambekk011@gmail.com,genf,blue


In [62]:
df = pd.merge(t, bc, on='email', how='left',indicator=True)
df["role"] = df["role_x"].fillna(df["role_y"])
df = df.drop(columns=["role_x","role_y",])

In [63]:
df.loc[df["_merge"] == "left_only"]

,person_id,display_name,email,team,id,custom_id,first_name,last_name,_merge,role
20,44236,casper joachim de kort,casper.joachim@icloud.com,blue,NaN,NaN,NaN,NaN,left_only,genf
93,<NA>,vebjørn risa,None,orange,NaN,NaN,NaN,NaN,left_only,mentor
109,44301,leander schumacher,leanderschumacher@outlook.com,pink,NaN,NaN,NaN,NaN,left_only,genf
125,<NA>,ole-andreas gjøsund,,pink,NaN,NaN,NaN,NaN,left_only,mentor
138,46614,emilie neumüller,46614@replacewithrealemail.com,yellow,NaN,NaN,NaN,NaN,left_only,genf
145,46628,rasmus nevanlinna,rasmusnevanlinna@gmail.com,yellow,NaN,NaN,NaN,NaN,left_only,genf
157,<NA>,susanne høines,None,yellow,NaN,NaN,NaN,NaN,left_only,mentor


In [64]:
df2 = pd.merge(t.dropna(subset=['person_id']), bc.dropna(subset=['custom_id']), left_on='person_id', right_on='custom_id', how='left',indicator=True)
df2 = df2.loc[df2["_merge"] == "both"]
df2["email"] = df2["email_x"].fillna(df2["email_y"])
df2.drop(columns = ["email_x","email_y"], inplace=True)
df2.head()

,person_id,display_name,role_x,team,id,custom_id,role_y,first_name,last_name,_merge,email
0,41647,michelle helen pfeifer,ansvarlig hjelpementor,blue,cff03a72-1567-482f-b2a3-7339612645cd,41647.0,member,Michelle Helén,Pfeifer,both,pfeifer.mhp@gmail.com
1,46663,emilio seth baumann,genf,blue,78dce177-fe9f-4c0a-925d-9662bdbed913,46663.0,member,Emilio Seth,Baumann,both,baumannemilio04@gmail.com
2,45966,william bekkevold,genf,blue,e5db7343-2c3f-47bb-9315-41e3b32e1387,45966.0,member,William,Bekkevold,both,williambekk011@gmail.com
3,44368,sophia jules pope,genf,blue,44722afe-34f7-4103-9db4-283cc6125a2f,44368.0,member,Sophia Jules,Pope,both,sophiajpope99@gmail.com
4,45366,jessica toader,genf,blue,6037deb2-f11c-451e-9b42-0c76f76edc2b,45366.0,member,Jessica,Toader,both,jessicatoader10@gmail.com


In [65]:
a = df.loc[df["_merge"] == "both", ["id","team","email"]]
b = df2.loc[:, ["id","team","email"]]
teams = pd.concat([a,b]).drop_duplicates()

In [72]:
t.loc[~t["email"].isin(teams["email"])]

,person_id,display_name,email,role,team
93,<NA>,vebjørn risa,None,mentor,orange
125,<NA>,ole-andreas gjøsund,,mentor,pink
138,46614,emilie neumüller,46614@replacewithrealemail.com,genf,yellow
157,<NA>,susanne høines,None,mentor,yellow


In [ ]:
missing = {"id" : ["9a3c278c-1d86-44bc-b3d5-b832bfa07204","a25912d9-1e44-40a0-aa35-ef06e1885408"], 
           "team"  : ["orange", "yellow"],
           #"email" : ,
           }
new = pd.DataFrame(missing.items(), columns=['id', 'team'])
teams = pd.concat([teams, new], ignore_index=True)

In [76]:
client.load_table_from_dataframe(teams.drop(columns=["email"],), 
                                 "members.teams_25_26",
                                 job_config=bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")).result()

LoadJob<project=genf-446213, location=EU, id=fe5ff690-9939-43d5-8fcb-7566a6fc2d45>

In [74]:
t

,person_id,display_name,email,role,team
0,41647,michelle helen pfeifer,pfeifer.mhp@gmail.com,ansvarlig hjelpementor,blue
1,<NA>,sigurd baltzersen,sigbaltz@hotmail.com,ansvarlig mentor,blue
2,<NA>,norah knutsen,norah.knutsen@outlook.com,ansvarlig mentor,blue
3,46663,emilio seth baumann,baumannemilio04@gmail.com,genf,blue
4,45966,william bekkevold,williambekk011@gmail.com,genf,blue
...,...,...,...,...,...
155,<NA>,ester muilwijk,esterimu@outlook.com,mentor,yellow
156,<NA>,sigvard bratlie,sigvard.bratlie@gmail.com,mentor,yellow
157,<NA>,susanne høines,None,mentor,yellow
158,<NA>,benjamin muilwjik,josef-benjamin@hotmail.com,mentor,yellow
